<a href="https://www.kaggle.com/code/alpy18/ab-testing-average-bidding-vs-maximum-bidding?scriptVersionId=113819093" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# İş Problemi
Facebook kısa süre önce mevcut "maximumbidding" adı verilen
teklif verme türüne alternatif olarak yeni bir teklif türü olan
"average bidding"’i tanıttı.
Müşterilerimizden biri olan bombabomba.com, bu yeni özelliği test
etmeye karar verdi ve average bidding'in maximumbidding'den
daha fazla dönüşüm getirip getirmediğini anlamak için bir A/B
testi yapmak istiyor.
A/B testi 1 aydır devam ediyor ve bombabomba.com şimdi sizden
bu A/B testinin sonuçlarını analiz etmenizi bekliyor.
Bombabomba.com için nihai başarı ölçütü Purchase'dır. Bu
nedenle, istatistiksel testler için Purchase metriğine
odaklanılmalıdır.

# Veri Seti Hikayesi
Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin yanı sıra
buradan gelen kazanç bilgileri yer almaktadır. Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır. Bu veri setleri
ab_testing.xlsx excel’inin ayrı sayfalarında yer almaktadır. Kontrol grubuna Maximum Bidding, test grubuna Average
Bidding uygulanmıştır.

Değişken | Açıklama
-|-
Impression | Reklam görüntüleme sayısı
Click | Görüntülenen reklama tıklama sayısı
Purchase | Tıklanan reklamlar sonrası satın alınan ürün sayısı
Earning | Satın alınan ürünler sonrası elde edilen kazanç

In [1]:
import pandas as pd
import statsmodels.stats.api as sms
from scipy.stats import shapiro, levene, ttest_ind, mannwhitneyu

In [2]:
control = pd.read_excel("/kaggle/input/bombabomba/ab_testing.xlsx", "Control Group")
test = pd.read_excel("/kaggle/input/bombabomba/ab_testing.xlsx", "Test Group")

In [3]:
control.head()  # Maximum Bidding

,Impression,Click,Purchase,Earning
0,82529.459271,6090.077317,665.211255,2311.277143
1,98050.451926,3382.861786,315.084895,1742.806855
2,82696.023549,4167.965750,458.083738,1797.827447
3,109914.400398,4910.882240,487.090773,1696.229178
4,108457.762630,5987.655811,441.034050,1543.720179


In [4]:
test.head()  # Average Bidding

,Impression,Click,Purchase,Earning
0,120103.503796,3216.547958,702.160346,1939.611243
1,134775.943363,3635.082422,834.054286,2929.405820
2,107806.620788,3057.143560,422.934258,2526.244877
3,116445.275526,4650.473911,429.033535,2281.428574
4,145082.516838,5201.387724,749.860442,2781.697521


In [5]:
control.shape

(40, 4)

In [6]:
test.shape

(40, 4)

In [7]:
control.isnull().sum()

Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64

In [8]:
test.isnull().sum()

Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64

In [9]:
control.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.0,101711.449068,20302.157862,45475.942965,85726.690349,99790.701078,115212.816543,147539.336329
Click,40.0,5100.657373,1329.985498,2189.753157,4124.304129,5001.220602,5923.803596,7959.125069
Purchase,40.0,550.894059,134.108201,267.028943,470.095533,531.206307,637.957088,801.795020
Earning,40.0,1908.568300,302.917783,1253.989525,1685.847205,1975.160522,2119.802784,2497.295218


In [10]:
test.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.0,120512.411758,18807.448712,79033.834921,112691.970770,119291.300775,132050.578933,158605.920483
Click,40.0,3967.549761,923.095073,1836.629861,3376.819024,3931.359804,4660.497911,6019.695079
Purchase,40.0,582.106097,161.152513,311.629515,444.626828,551.355732,699.862360,889.910460
Earning,40.0,2514.890733,282.730852,1939.611243,2280.537426,2544.666107,2761.545405,3171.489708


In [11]:
sms.DescrStatsW(control["Purchase"]).tconfint_mean()  # Kontrol grubu güven aralığı

(508.0041754264924, 593.7839421139709)

In [12]:
sms.DescrStatsW(test["Purchase"]).tconfint_mean()  # Test grubu güven aralığı

(530.5670226990062, 633.6451705979289)

In [13]:
test["Group"] = "T"
control["Group"] = "C"

df = pd.concat([control, test])
df.reset_index(inplace=True, drop=True)
df.head()

,Impression,Click,Purchase,Earning,Group
0,82529.459271,6090.077317,665.211255,2311.277143,C
1,98050.451926,3382.861786,315.084895,1742.806855,C
2,82696.023549,4167.965750,458.083738,1797.827447,C
3,109914.400398,4910.882240,487.090773,1696.229178,C
4,108457.762630,5987.655811,441.034050,1543.720179,C


In [14]:
df.tail()

,Impression,Click,Purchase,Earning,Group
75,79234.911929,6002.213585,382.047116,2277.863984,T
76,130702.239410,3626.320072,449.824592,2530.841327,T
77,116481.873365,4702.782468,472.453725,2597.917632,T
78,79033.834921,4495.428177,425.359102,2595.857880,T
79,102257.454089,4800.068321,521.310729,2967.518390,T


**Hipotezler**

H0: M1 = M2

Average Bidding ve Maximum Bidding'in dönüşüm oranı arasında istatistiksel olarak anlamlı bir fark yoktur.


H1: M1 != M2

Average Bidding ve Maximum Bidding'in dönüşüm oranı arasında istatistiksel olarak anlamlı bir fark vardır.

**Kazanç Ortalamaları**

In [15]:
df.groupby("Group").agg({"Purchase": ["mean", "median"]})

Purchase            
             mean      median
Group                        
C      550.894059  531.206307
T      582.106097  551.355732

Ortalamalara bakıldığında kontrol grubu olan maximum bidding yönteminin, test grubu olan average bidding'den daha az satın alma getirdiği görülüyor.

**Normallik Varsayımı**

H0: Normal dağılım varsayımı sağlanmaktadır.

H1: Normal dağılım varsayımı sağlanmamaktadır.

p < 0.05 ise H0 reddedilir, p > 0.05 ise H0 reddedilmez.

In [16]:
# Kontrol grubu için
test_stat, pvalue = shapiro(df.loc[df["Group"] == "C", "Purchase"])
print("Test Stat: %.4f, p-value: %.4f" % (test_stat, pvalue))

Test Stat: 0.9773, p-value: 0.5891


In [17]:
# Test grubu için
test_stat, pvalue = shapiro(df.loc[df["Group"] == "T", "Purchase"])
print("Test Stat: %.4f, p-value: %.4f" % (test_stat, pvalue))

Test Stat: 0.9589, p-value: 0.1541


İki grubun da p değerleri 0.05'ten büyük olduğu için H0 reddedilemez. 

Böylece normal dağılım varsayımı sağlanmış olur.

**Varyans Homojenliği**

H0: Varyanslar homojendir.

H1: Varyanslar homojen değildir.

p < 0.05 ise H0 reddedilir, p > 0.05 ise H0 reddedilmez.

In [18]:
test_stat, pvalue = levene(df.loc[df["Group"] == "C", "Purchase"],
                           df.loc[df["Group"] == "T", "Purchase"])

print("Test Stat: %.4f, p-value: %.4f" % (test_stat, pvalue))

Test Stat: 2.6393, p-value: 0.1083


p değeri 0.05'ten büyük olduğu için H0 reddedilemez.

Dolayısıyla varyans homojenliği sağlanmış olur.


**Bağımsız İki Örneklem T Testi**

İki varsayım da sağlandığı için bağımsız iki örneklem t testi (parametrik test) kullanılır.

In [19]:
test_stat, pvalue = ttest_ind(df.loc[df["Group"] == "C", "Purchase"],
                              df.loc[df["Group"] == "T", "Purchase"])

print("Test Stat: %.4f, p-value: %.4f" % (test_stat, pvalue))

Test Stat: -0.9416, p-value: 0.3493


**Sonuç**

p değeri 0.05'ten büyük olduğu için H0 reddedilemez.

Sonuç olarak Average Bidding ve Maximum Bidding'in dönüşüm oranı arasında istatistiksel olarak anlamlı bir fark olmadığı kabul edilir.

**Sonuçların Analizi**
* Karşılaştırmada iki grup olduğu için ve bu grupların ortalamalarını karşılaştırmak daha mantıklı bir seçenek olacağı için varsayımlarımızını sağlanıp sağlanmamasına göre t testini veya mannwhitneyu testini kullanmamız gerekiyordu. İki varsayımımızın da sağlanıyor olmasından dolayı bu hipotezde t testini kullandık.

* Elde edilen satın alma sayılarını, Maximum Bidding ve Average Bidding yöntemlerine göre ortalamalarını alarak ayrı ayrı istatiksel olarak t testini kullanarak inceledik. Ve Average Bidding kullanıldığında satın alma açısından istatistsel olarak anlamsal bir fark oluşmadığı sonucuna vardık. Bu nedenle Average Bidding yönteminin kullanılmasının şirkete artı bir getirisi olmayacak, bu teklif türünü kullanıma getirmek için edilecek masrafa değmeyecektir.